https://lilianweng.github.io/lil-log/2017/07/08/predict-stock-prices-using-RNN-part-1.html
    

# Using Stock Market Data to Make Price Predictions
I'm fascinated by the stock market, and I'm learning how to work with neural networks. This is a project I've been working on in my own time, in the hopes I can eventually use it to make some money.

By using data from twelvedata.com's stock API, I implement the keras library's neural networks to make stock predicitons. 
## Methods
My thoughts on how to best prepare data for the neural network
### Inputs/Outputs Ideas
1. Input raw prices, output raw price predictions
2. Input price derivatives, output percentage growth at different times
3. Input average price over surrounding time periods, output raw price predictions

### Secondary Objectives
1. Input other stock indicators such as P/E, EMACS, etc.


## Importing Libraries

In [2]:
# Import necessary libraries
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from tensorflow.keras import layers
import sklearn as sk
import requests as rq
import datetime as dt
import json
from os import path, mkdir
from random import choice, sample


## Helper Functions

In [7]:
def getLocalApiKey(api):
    """Returns my personal api key from a private directory with format:
    {
    "apiKeys": 
        {
            "twelvedata": "apikeyString"
        }
    }
    """
    with open("../../../Private/apiKeys.txt") as f:
        apiDict = json.load(f)
        apiKey = apiDict["apiKeys"][api]
        return apiKey


def print_dict_tree(d, indent=0):
    """Print tree of keys in `dict` object.
    
    Prints the different levels of nested keys in a `dict` object. When there
    are no more dictionaries to key into, prints objects type and byte-size.

    Input
    -----
    d : dict
    """
    for key, value in d.items():
        print('    ' * indent + str(key), end=' ')
        if isinstance(value, dict):
            print(); print_dict_tree(value, indent+1)
        else:
            print(":", str(type(d[key])).split("'")[1], "-", str(len(str(d[key]))))

            
def getStock(apiKey: str, symbol: str, period: str, start_date=None, end_date=None, dp=3):
    '''Returns the json of stock symbol
    
    Requests stock values between start and end date. 
    If no start and end date are selected, it automatically chooses dates 
    from 1 year and 2 months ago.
    
    Input
    -----
    apiKey: your api key
    symbol: stock symbol
    period: the time interval for stock data. Check twelvedata.com for valid inputs
    start_date: yyyy-mm-dd
    end_date: yyyy-mm-dd
    '''
    if not start_date:
        start_date = dt.datetime.today() - dt.timedelta(days=365)
        start_date = str(start_date.strftime('%Y-%m-%d'))
    if not end_date:
        end_date = dt.datetime.today() - dt.timedelta(days=60)
        end_date = str(end_date.strftime('%Y-%m-%d'))
    
    # make API request
    ts = rq.get("https://api.twelvedata.com/time_series?symbol="+symbol+"&interval="+period+"&start_date="+start_date+"&end_date="+end_date+"&dp="+str(dp)+"&apikey="+apiKey).json()
    # Check to see if it is a valid time series. Incorrect values for the 
    # interval or start/end_dates will still return dictionaries, but they will be invalid.
    assert(ts["values"])
    return ts


def saveTS(ts):
    '''Saves a json dict of a stock time series as a .txt
    
    Will create a folder to put saved stock time series in. Future timeseries
    will be placed in this folder as well.
    
    Input
    -----
    ts: timeseries that you want saved
    '''
    symbolName = ts["meta"]["symbol"].replace("/", "")
    if not path.isdir('stockTimeSeries'):
        mkdir("stockTimeSeries")
    with open("stockTimeSeries/"+symbolName+'.txt', 'w') as outfile:
        json.dump(ts, outfile)
        
        
def formatDataDailyPrice(ts: dict, backSearch=90, forecastPattern: list = [1, 5, 10, 20, 50]):
    '''Returns time series in a format that is usable by the keras neural network
    
    Returns a list of prices 
    '''
    
    # Ordered list of prices from most old to recent
    prices = [period["close"] for period in ts["values"]]
    prices.reverse()
    
    #print(prices)
    
    
    # Prevents trying to verify prices in unknown time periods
    nPrices = len(prices)
    gap = max(forecastPattern)
    maxIndex = nPrices - gap
    minIndex = backSearch
    indices = [i for i in range(minIndex, maxIndex)]
        
    X = []
    y = []
    
    # Allows us to add prices with their corresponding forecasts to new lists in a random order
    while indices:
        index = choice(indices)
        indices.remove(index)
        xBackSearch = []
        for i in range(backSearch):
            xBackSearch.append(float(prices[index-i]))
        X.append(xBackSearch)
        yForecast = []
        for delta in forecastPattern:
            yForecast.append(float(prices[index+delta]))
        y.append(yForecast)
    
    X = np.reshape(X, (-1, backSearch))
    y = np.reshape(y, (-1, len(forecastPattern)))
    # Prints price, followed by forecast
    
    return X, y
    

def splitData(X: list, y: list, ratio=.5):
    '''Returns 4 lists, composed to training and test data
    
    Input
    -----
    X: data for input neurons
    y: data for results
    
    Output
    ------
    '''
    split = int(len(X)*ratio)
    x_training = X[:split]
    y_training = y[:split]
    x_test = X[split:]
    y_test = y[split:]
    return x_training, y_training, x_test, y_test
    

def trainData(x_train, y_train, scalar=False, priceTiming="cls"):
    '''
    descriptioin
    
    Input
    -----
    priceTiming: Can be "avg", which averages closing price over surrounding time periods, or
    "cls", which uses closing price for that time period
    To implement: Averaging over course of day, averaging over course of weeks, etc
    
    Output
    ------
    X: A list of lists containing stock prices from the last 
    90 entries from an arbitrary point in time
    y: a list of lists containing the stock prices a number of 
    days in the future corresponding to theforecast pattern'''
    pass

## Using AAPL stock to train the neural network
It is uncertain whether all stocks behave in the same way. For example, a neural network learning from apple stock 
may not be able to be as accurate when predicting oil stocks, or even other tech sector stocks. However, in order to create the foundation of the neural network, I wanted to use it on a high volume, high market cap, relatively low volatility stock. If I can train it with AAPL stock, I may be able to apply the neural network to other stocks.

## Model 1
In model 1, I use raw data to make predictions. Even making predictions only 3 days in advance gives poor results.

In [4]:
# get and save the aapl stock time series as a .txt
myApiKey = getLocalApiKey("twelvedata")

aaplTS = getStock(myApiKey, "aapl", "1day", start_date="2002-01-01", end_date="2020-9-30")

X, y = formatDataDailyPrice(aaplTS, backSearch=300, forecastPattern=[1, 2, 3])
x_train, y_train, x_test, y_test = splitData(X, y)
print(x_train[:10])
print(y_train[:10])


[[80.967 81.085 79.423 ... 44.195 44.245 46.465]
 [ 6.176  6.082  5.948 ...  6.076  6.006  6.239]
 [ 4.732  4.74   4.717 ...  4.347  4.465  4.64 ]
 ...
 [ 8.593  8.555  8.517 ...  3.219  3.321  3.364]
 [ 7.259  7.249  7.195 ...  5.417  5.64   5.721]
 [ 2.716  2.725  2.656 ...  0.988  0.955  0.937]]
[[ 77.378  77.165  79.713]
 [  6.112   6.163   6.149]
 [  4.609   4.614   4.628]
 [  0.375   0.387   0.373]
 [ 28.88   29.93   28.372]
 [  6.508   6.388   6.422]
 [113.902 111.113 112.738]
 [  8.57    8.635   8.653]
 [  7.214   7.302   7.38 ]
 [  2.888   2.996   3.01 ]]


In [5]:
model = Sequential()
model.add(Dense(50, input_dim=len(x_train[0]), activation = 'relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(len(y_train[0]), activation='softmax'))

model.compile(optimizer='sgd',
              loss='mse',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=500, batch_size=len(x_train)//10)

score = model.evaluate(x_test, y_test, batch_size=len(x_train)//10)
print("accuracy:", score[1])

Epoch 1/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2598 - accuracy: 0.3348
Epoch 2/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2925 - accuracy: 0.3860
Epoch 3/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2452 - accuracy: 0.3416
Epoch 4/500
11/11 [==============================] - 0s 2ms/step - loss: 858.2373 - accuracy: 0.3457
Epoch 5/500
11/11 [==============================] - 0s 2ms/step - loss: 858.2123 - accuracy: 0.3475
Epoch 6/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2186 - accuracy: 0.3403
Epoch 7/500
11/11 [==============================] - 0s 2ms/step - loss: 858.2253 - accuracy: 0.3407
Epoch 8/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2383 - accuracy: 0.3747
Epoch 9/500
11/11 [==============================] - 0s 2ms/step - loss: 858.2124 - accuracy: 0.3620
Epoch 10/500
11/11 [==============================] - 0s 2ms/step - loss: 858.2123 - accura

11/11 [==============================] - 0s 1ms/step - loss: 858.2055 - accuracy: 0.3688
Epoch 161/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2057 - accuracy: 0.3670
Epoch 162/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2064 - accuracy: 0.3715
Epoch 163/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2056 - accuracy: 0.3729
Epoch 164/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2055 - accuracy: 0.3792
Epoch 165/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2061 - accuracy: 0.3539
Epoch 166/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2084 - accuracy: 0.3833
Epoch 167/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2054 - accuracy: 0.3684
Epoch 168/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2081 - accuracy: 0.3172
Epoch 169/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2057 - a

11/11 [==============================] - 0s 972us/step - loss: 858.2055 - accuracy: 0.3725
Epoch 319/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2050 - accuracy: 0.3525
Epoch 320/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2051 - accuracy: 0.3715
Epoch 321/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2061 - accuracy: 0.3194
Epoch 322/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2051 - accuracy: 0.3815
Epoch 323/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2050 - accuracy: 0.3851
Epoch 324/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2049 - accuracy: 0.3960
Epoch 325/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2046 - accuracy: 0.3883
Epoch 326/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2055 - accuracy: 0.3652
Epoch 327/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2057 -

11/11 [==============================] - 0s 1ms/step - loss: 858.2052 - accuracy: 0.3566
Epoch 477/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2054 - accuracy: 0.3675
Epoch 478/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2051 - accuracy: 0.3820
Epoch 479/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2093 - accuracy: 0.3638
Epoch 480/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2073 - accuracy: 0.3584
Epoch 481/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2065 - accuracy: 0.3761
Epoch 482/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2057 - accuracy: 0.3657
Epoch 483/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2059 - accuracy: 0.3847
Epoch 484/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2052 - accuracy: 0.3797
Epoch 485/500
11/11 [==============================] - 0s 1ms/step - loss: 858.2056 - a

### Model 1 Commentary
Despite a fairly deep neural network and training on data representing the last 100 days, the neural network struggles to find much pattern in the stock data. However, this may be explained by 

In [5]:
def formatDataDailyChange(ts: dict, backSearch=90, forecastPattern: list = [1, 5, 10, 20, 50]):
    '''Returns time series in a format that is usable by the keras neural network.
    
    
    X: [[p0, p1, p2,.. p_backSearch], [etc]]
    y: [[p_backSearch+forecastPattern_1, p_backSearch+forecastPattern_2,... p_backSearch+forecastPattern_n], [etc]]
    '''
    
    priceChanges = getDailyChanges(ts)
    
    # Prevents trying to verify prices in unknown time periods
    nPrices = len(priceChanges)
    gap = max(forecastPattern)
    maxIndex = nPrices - gap
    minIndex = backSearch
    indices = [i for i in range(minIndex, maxIndex)]
    
    X = []
    y = []
    
    # Allows us to add prices with their corresponding forecasts to new lists in a random order
    while indices:
        index = choice(indices)
        indices.remove(index)
        xBackSearch = []
        for i in range(backSearch):
            xBackSearch.append(float(priceChanges[index-i]))
        X.append(xBackSearch)
        yForecast = []
        for delta in forecastPattern:
            yForecast.append(float(priceChanges[index+delta]))
        y.append(yForecast)
    
    X = np.reshape(X, (-1, backSearch))
    y = np.reshape(y, (-1, len(forecastPattern)))
    # Prints price, followed by forecast
    
    return X, y
    
        
def getDailyChanges(ts: dict):
    '''Returns list of all prices as their daily percentage change from one day to the next
    
    Input
    -----
    prices
    
    Returns: a list of price changes
    '''
    # Ordered list of prices from most old to recent
    prices = [float(period["close"]) for period in ts["values"]]
    prices.reverse()
    
    #Transform prices into priceChanges
    priceChanges = []
    for index, price in enumerate(prices[1:]):
        priceChange = price / prices[index] # index is 1 element behind since we started enumerate at second price
        priceChanges.append(priceChange)

    return priceChanges


NameError: name 'aaplTS' is not defined

In [9]:
myApiKey = getLocalApiKey("twelvedata")

aaplTS = getStock(myApiKey, "aapl", "1day", start_date="2002-01-01", end_date="2020-9-30")

X, y = formatDataDailyChange(aaplTS, backSearch=10, forecastPattern=[1])
x_train, y_train, x_test, y_test = splitData(X, y)

In [10]:
model = Sequential()
model.add(Dense(50, input_dim=len(x_train[0]), activation = 'relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(len(y_train[0]), activation='softmax'))

model.compile(optimizer='sgd',
              loss='mse',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=len(x_train)//100)

score = model.evaluate(x_test, y_test, batch_size=len(x_train)//10)
print("accuracy:", score[1])

Epoch 1/100
103/103 [==============================] - 0s 597us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 2/100
103/103 [==============================] - 0s 561us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 3/100
103/103 [==============================] - 0s 567us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 4/100
103/103 [==============================] - 0s 569us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 5/100
103/103 [==============================] - 0s 572us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 6/100
103/103 [==============================] - 0s 582us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 7/100
103/103 [==============================] - 0s 622us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 8/100
103/103 [==============================] - 0s 611us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 9/100
103/103 [==============================] - 0s 616us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 10/100
103/103 [===============

103/103 [==============================] - 0s 549us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 77/100
103/103 [==============================] - 0s 543us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 78/100
103/103 [==============================] - 0s 556us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 79/100
103/103 [==============================] - 0s 541us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 80/100
103/103 [==============================] - 0s 547us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 81/100
103/103 [==============================] - 0s 541us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 82/100
103/103 [==============================] - 0s 543us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 83/100
103/103 [==============================] - 0s 550us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 84/100
103/103 [==============================] - 0s 546us/step - loss: 4.6988e-04 - accuracy: 0.0089
Epoch 85/100
103/103 [===================

## Model 2 Commentary
Something is wrong here. I need to modify my x and y training data so that accuracy actually means something. It would also be helpful to be able to visualize the data. I'm going to try to implement an RNN, and use some different methods for formatting my data.

In [ ]:
def convert_to_3d_matrix(X, y):
    for i, x in enumerate(X):
        new_list = [[j] for j in x]
    

In [13]:
# each example (of which we have a 100) is a sequence of 10 words and
# each words is encoded as 16 element vectors

X = np.random.rand(100, 5, 1) #(total_length, back_search_length, 1)
y = np.random.choice(1, 100)



print(X)

model = Sequential()
model.add(LSTM(128, input_shape=(5, 1)))
model.add(Dense(2, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='sgd')

# fit model
model.fit(X, y, epochs=3, batch_size=3)

[[0.99873491 1.01352139 1.06496462 0.99309636 0.98662072 0.9922359
  0.99471789 1.00088113 1.00040067 1.01064993]
 [1.01399966 0.9553376  0.97703334 1.0044547  1.01534609 0.98355577
  0.99071305 1.00118194 0.98708875 1.00124601]]
Epoch 1/3


ValueError: in user code:

    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib